Top-Down Archeology with a Geometric Language
========================

In [ ]:
%config IPCompleter.greedy=True

In [1]:
from pyplasm import *
from math import sin, cos

Evaluating fenvs.py..
...fenvs.py imported in 0.009412 seconds


The column is assembled by gathering the parts of the object, and by putting each part on the top of the previous ones. In particular, it is composed of a vertical cylndr that is wider at the bottom, two geometric toruses, torus bot at the bottom, and torus top at the top of the cylinder, respectively, by two parallelepiped bases base and base top, and by the capital , generated by rotated union of two squared baskets. Truncone and Torus are the Plasm primitives for generation of a truncated cone and a torus surface, respectively. The composition with the Join operator transforms their output from a surface to its solid convex hull.

In [2]:
def columna(dm, h, h_base):
    """
    Columna (1.5, 7., 1.)
    Note: Columna (1, 7, 1) will produe an equivalent result since those parameters will be casted to float (necessary, see below)
    
    :param dm: is the circumference diameter at the column basis;
    :param h: is the column height;
    :return: pyplasm.xgepy.Hpc
    """
    dm = float(dm)
    h = float(h)
    h_base = float(h_base)
    
    cylndr = COMP([JOIN, TRUNCONE([dm/2, .8*dm/2, h])])(24)
    torusBot = COMP([JOIN, TORUS([dm/12, dm/2])])([8, 27])
    torusTop = COMP([JOIN, TORUS([.8*(dm/12), .8*(dm/2)])])([8, 24])
    base = COMP([T([1, 2])([7*dm/-12, 7*dm/-12]), CUBOID])([7*dm/6,7*dm/6, h_base])
    baseTop = COMP([T([1, 2])([7*dm/-12, 7*dm/-12]), CUBOID])([7*dm/6, 7*dm/6, dm/6])
    capital = SUM([COMP([JOIN, TRUNCONE([.8*dm/2, 1.2*dm/2, h/8])])(4),
        COMP([R([1, 2])(PI/4), JOIN, TRUNCONE([.8*dm/2, 1.2*dm/2, h/8])])(4)])
    return TOP([TOP([TOP([TOP([TOP([base, torusBot]), cylndr]), torusTop]), capital]), baseTop])

In [3]:
c = T([1, 2])([0.9, 0.9])(columna(1.5,7.,1.))
travertine = "#E2DECD"
VIEW(HEX(travertine)(c))

Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10886fb10> >

![columna](./images/columna.png)

![top](./images/top.png)

![capital](./images/capital.png)

### float(n) explained
dm, h, h_base need to be double/float otherwise they could be rounded to 0 cousing a Segmentation Fault 11. (Reported on my machine) 

In [4]:
def columnas(obj,n,spacing):
    """
    columnas(column, 4, 7.4)
    :param obj: 
    :param n: 
    :param spacing: 
    :return: 
    """
    return STRUCT(NN(n)([obj, T(1)(spacing)]))

In [7]:
myCol = T([1, 2])([0.9, 0.9])(columna(1.5,7.,1.))
myColumns = columnas(myCol,6,7./4)
VIEW(myColumns)

Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10e4ee810> >

![columnas](./images/columnas.png)

In [9]:
VIEW(STRUCT([columnas(c,4,7./4),  T(2)(7./2)(columnas(c,4,7./4))]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10e4f0330> >

![columnas 2 rows](./images/columnas_2.png)

![columnas 3 rows](./images/columnas_3.png)

# Arch

In [19]:
def ArchSurface(rr, w, h):
    """
    ArchSurface(5,1,1)
    :param rr: outerRadius
    :param w: innerRadius
    :param h: height (necessary)
    :return: pyplasm.xgepy.Hpc
    """
    
    x1 = lambda u: rr*cos(u[0])
    y1 = lambda u: rr*sin(u[0])
    x2 = lambda u: (rr-w)*cos(u[0])
    y2 = lambda u: (rr-w)*sin(u[0])
    z = lambda u: h
    
    bz11 = BEZIER(S1)([CONS([x1,y1,z])])
    bz12 = BEZIER(S1)([CONS([x2,y2,z])])
    return BEZIER(S2)([bz11, bz12])

In [23]:
def Arch(length, w, depth ,angle):
    """
    Arch(10, 1, 1, 0.5*PI)
    :param length: length of the arch, it's the horizontal axis
    :param w: thickness in terms of outerRadius-innerRadius
    :param depth: depth of the arch
    :param angle: angle wich the arch embraces
    :return: pyplasm.xgepy.Hpc
    """
    radius = (length/2) / cos(angle/2)
    domain2D = PROD([T(1)(angle/2)(INTERVALS(PI-angle)(16)), QUOTE([1])])
    domain3D = PROD([domain2D, QUOTE([1])])
    ArchSurf2D_0 = ArchSurface(radius,w,0)
    ArchSurf2D_1 = ArchSurface(radius,w,depth)
    SolidMap = BEZIER(S3)([ArchSurf2D_0, ArchSurf2D_1])
    return MAP(SolidMap)(domain3D)

In [24]:
myArch = Arch(10, 1, 1, 0.5*PI)
VIEW(myArch)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x117a85780> >

![arch](./images/arch.png)